In [13]:
import os
from glob import glob
import shutil
from tqdm.notebook import tqdm 
import dicom2nifti
import numpy as np
import nibabel as nib
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism

In [14]:
training_directory="D:/MLDatasets/TCGA-GBM Datasets/Training"

In [17]:
flair_files = sorted(glob(os.path.join(training_directory, f"images_{0}", "*.nii.gz")))
t1Gd_files = sorted(glob(os.path.join(training_directory, f"images_{1}", "*.nii.gz")))
t2_files = sorted(glob(os.path.join(training_directory, f"images_{2}", "*.nii.gz")))
mask_files = sorted(glob(os.path.join(training_directory, f"segmentation", "*.nii.gz")))
print(len(flair_files))
print(len(t1Gd_files))
print(len(t2_files))
print(len(mask_files))

102
102
102
102


In [19]:
for i in range(len(flair_files)):
    flair_image = nib.load(flair_files[i]).get_fdata()
    t1Gd_image = nib.load(t1Gd_files[i]).get_fdata()
    t2_image = nib.load(t2_files[i]).get_fdata()
    mask = nib.load(mask_files[i]).get_fdata()
    mask = mask.astype(np.uint8)
    mask[mask==4] = 3

    combined_image = np.stack([flair_image, t1Gd_image, t2_image], axis=3)
    
    np.save(os.path.join(training_directory, "imagesv1", f"image_{i}.npy"), combined_image)
    np.save(os.path.join(training_directory, "segmentationv1", f"mask_{i}.npy"), mask)

In [30]:
import os
from glob import glob
import shutil
from tqdm import tqdm
import dicom2nifti
import numpy as np
import nibabel as nib
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism


def prepare(in_dir, pixdim=(1.0, 1.0, 1.0), a_min=0, a_max=503, spatial_size=[128,128,144], trainloader_batchsize=1):
    print("Preparing...")
    set_determinism(seed=0)

    path_train_volumes = sorted(glob(os.path.join(in_dir, "imagesv1", "*.npy")))
    path_train_segmentation = sorted(glob(os.path.join(in_dir, "segmentationv1", "*.npy")))

    '''DEBUG'''
    print(f"Length of path_train_volumes: {len(path_train_volumes)}")    
    print(f"Length of path_train_segmentation: {len(path_train_segmentation)}")    
    '''DEBUG'''

    train_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_train_volumes, path_train_segmentation)]
    
    train_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True), 
            CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),   
            ToTensord(keys=["vol", "seg"]),

        ]
    )

    train_ds = CacheDataset(data=train_files, transform=train_transforms,cache_rate=1.0)
    print("attempted cache")
    train_loader = DataLoader(train_ds, batch_size=trainloader_batchsize)
    print("Preparing done.")

    return train_loader



In [31]:

trainloader = prepare(training_directory)

Preparing...
Length of path_train_volumes: 102
Length of path_train_segmentation: 102


Loading dataset:   0%|          | 0/102 [00:00<?, ?it/s]


RuntimeError: applying transform <monai.transforms.spatial.dictionary.Spacingd object at 0x0000024A24298D30>